In [1]:
import json

In [2]:
with open('SelectedSecurity.json') as json_file:
    hkmaisin = json.load(json_file)

In [3]:
def transformYear(b):
    b['original_maturity'] = int(b['original_maturity'].split("-")[0])
    return b
hkmaisin_=list(map(lambda b: transformYear(b),hkmaisin))

In [4]:
from collections import defaultdict
TenorVsBond = defaultdict(list)
for b in hkmaisin_:
    TenorVsBond[ b['original_maturity'] ].append(b)

In [5]:
import random
import numpy as np
import uuid
import re
from typing import Dict
import time

class PrimaryInvestorRobot:
    def __init__ (self,name:str,TenorVsBond:Dict[int, Dict], tenorWeight:Dict[int, float], notional:int, std:float, bidask="ASK", ccy="HKD"):
        self.name = name
        self.TenorVsBond = TenorVsBond
        self.tenorWeight = tenorWeight
        self.notional = notional
        self.bidask = bidask
        self.std = std
        self.ccy = "HKD"
        self.YMDMatcher = re.compile(r"(\d{4})-(\d{2})-(\d{2})")
    def generateTrade(self):
        tenor = random.choices(list(tenorWeight.keys()), weights=list(tenorWeight.values()), k=1)[0]
        tradeid = str(uuid.uuid4())
        trade={}
        asset={}

        trade["id"] = tradeid
        trade["tradeType"] = "BOND"
        trade["timestamp"] = int(time.time() * 1000)
        security = random.choice(list(self.TenorVsBond[tenor]))
        
        mDate = self.YMDMatcher.match(security['expected_maturity_date'])
        if mDate is not None:
            year = int(mDate.group(1)) - tenor
            trade["tradeDate"] = f"{year}-{mDate.group(2)}-{mDate.group(3)}"
        
        asset['securityId'] = security['isin_code']
        asset['currency'] = self.ccy
        asset['price'] = float(security["coupon"])
        asset["bidask"] = self.bidask
        asset["notional"] = int(np.round(np.abs(np.random.normal(0, self.std, 1)), -3)[0])+self.notional
        trade['asset'] = asset
        
        return trade

In [6]:
tenorWeight = {15:0.3, 10:0.5, 5:0.2}
primaryInvestorRobot = PrimaryInvestorRobot("abcd",TenorVsBond,tenorWeight, 10000000,10000000)
sampleTrade = primaryInvestorRobot.generateTrade()

In [7]:
sampleTrade

{'id': 'bae8fa88-1470-4681-9718-2716229d2bf1',
 'tradeType': 'BOND',
 'timestamp': 1580655986329,
 'tradeDate': '2011-08-04',
 'asset': {'securityId': 'HK0000085537',
  'currency': 'HKD',
  'price': 2.46,
  'bidask': 'ASK',
  'notional': 10312000}}

## write to avro file
>Reference : http://layer0.authentise.com/getting-started-with-avro-and-python-3.html

In [8]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [9]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

In [10]:
schemaFile = "bondtrade.avsc"
avroFile = "bondtrade.avro"

In [11]:
schema = avro.schema.Parse(open(schemaFile, "rb").read())

In [12]:
writer = DataFileWriter(open(avroFile, "wb"), DatumWriter(), schema)

In [13]:
writer.append(sampleTrade);

NameError: name 'trade' is not defined

In [ ]:
writer.close()

In [ ]:
#reader = DataFileReader(open(avroFile, "rb"), DatumReader())
#for bondtrade in reader:
#    print (bondtrade)
#reader.close()

In [ ]:
uuid.uuid4()